# Diffie Hellman

First define the imports

In [58]:
# Required modules
from cryptography import hazmat
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat, load_pem_parameters, load_pem_public_key, load_pem_private_key

# Now the hash functions

In [59]:
def hash_data_sha256(data):
    # Create a SHA256 hash object
    digest = hashes.Hash(hashes.SHA256())
    
    # Update the hash object with the data to be hashed
    digest.update(data)
    
    # Finalize the hash and get the digest
    hash_digest = digest.finalize()
    
    return hash_digest

hash_data_sha256(b"Hello World!")

b'\x7f\x83\xb1e\x7f\xf1\xfcS\xb9-\xc1\x81H\xa1\xd6]\xfc-K\x1f\xa3\xd6w(J\xdd\xd2\x00\x12m\x90i'

# Figure out how Serialisation works
From my understanding of the serialization module, it exposes functions to serialise and deserialise keys and certificates.

It also exposes constants for encoding and decoding.

In [60]:

def read_private_key():
    # random private key is generate with openssl in pem format using
    # openssl genpkey -algorithm RSA -out private_key.pem -pkeyopt rsa_keygen_bits:2048
    # Serialize the data to PEM format

    with open("private_key.pem", "rb") as key_file:
        data = key_file.read()
    return load_pem_private_key(data, password=None)
    
    # return serialized_data


# Testing encryption
OAEP (Optimal Asymmetric Encryption Padding) is a padding scheme that makes encryption much more secure. 
It uses a hash function (in this case, SHA224) to process the data before encryption, 
making it harder for attackers to exploit predictable patterns in encrypted messages.
## Padding
OAEP padding is public information, not part of the keys. All parties know about this

In [61]:
from cryptography.hazmat.primitives.asymmetric import padding
# dw this is just for testing out things, im not using it for anything else.

padding = padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA224()), algorithm=hashes.SHA224(), label=None)
encrypted = read_private_key().public_key().encrypt(b"Hello World!", padding=padding)
print(encrypted)
read_private_key().decrypt(encrypted, padding=padding)


b'q\xd5\xf4c`\xc0Gz\x92=\x02\x1eI\xc8\xf0\x82\x14\x80\xca\x9f-\xb4\x8e\xfe\xe6\xd4zr\xcc\xfa\xdd!\xa2\x7f\xbd\xa7\xc6\xcf\xee\x1a:\xb6__?\xb6;\x04\xab\xf2\xd4\x8e\xfbWP\xac!\x8f\x92\xf29\x07+\xff\xf01\x10\xeb\x99\xf9)\x05\xbe3K\xf4"9\x88\xf0_Y:\xa6\xd5\xd8\xb3\xbd\xe2\x85\xb9\x8a\xbcJ\x19\x89\x8cN\x90\xa1\x8a\xecC\xccD\xb2\xebM\xbdI\x81M\xcb\xb3@Q"\xf6>\xe7\x1f\xd9\xad[0\x13\xfd%,\x07Sf\x9eZ\x0b\xf5v\xf2\x14-\x14\xf6\xf1\n{w\x86\xa0\xf9\xc8\xf1j"\x0e\xd7\xd3U\x12\xe7\x12I4\x17\xf7_\xbf\x02\\k\xe6)\x9f\xe2%lxTh\x05:cG\xec\x1ay\x0eC\x8a\xca$}\x0bL\xcba\xbb\\\x0c\x0e\xcfb1\x8c\xc1L@\x06Y\x97\xfd\x10\xd7\xc6)\x07\x9b\xaa\xfe\x07\x83\xdb\x92\xbc\x99\x7f\xdca>\'\x19\x88\xa9\x91S3-t,B \x9au3\x9fC\xfb\ni\xbf\xd7\xb5\xa5\xa6\xb39+'


b'Hello World!'

# Parameters for Diffie Hellman
Diffie Hellman key exchange requires the parameters to be the same before the key exchange can be performed.

This is why we need to modify the DiffieHellman() function to accept the parameters.

In [62]:

def serialize_public_key(public_key):
    return public_key.public_bytes(encoding=serialization.Encoding.PEM, format=serialization.PublicFormat.SubjectPublicKeyInfo)

def deserialize_public_key(pem):
    # Deserialize the public key received from the other party
    public_key = serialization.load_pem_public_key(pem)
    return public_key

def dh_params():
    # create diffie hellman parameters
    # key_size = 512 because it's faster to execute
    parameters = dh.generate_parameters(generator=2, key_size=512, backend=None)
    return parameters

# Had to change the name from DiffieHellman() to diffie_hellman() to follow the naming conventions
def diffie_hellman(parameters):
    # generate the public key
    print(parameters.parameter_numbers().g) # 2
    key_pair = parameters.generate_private_key()
    return key_pair, serialize_public_key(key_pair.public_key())


params = dh_params()
# Actor 1 generates their key pair and serializes the public key
alice_private_key, alice_public_pem = diffie_hellman(params)
print("Alice Public Key:", alice_public_pem)

# Bob generates their key pair and serializes the public key
bob_private_key, bob_public_pem = diffie_hellman(params)
print("Bob Public Key:", bob_public_pem)

# Eave generates their key pair and serializes the public key
# params are public, so eave also has them 
eave_private_key, eave_public_pem = diffie_hellman(params)
print("Eave Public Key:", eave_public_pem)


2
Alice Public Key: b'-----BEGIN PUBLIC KEY-----\nMIGbMFMGCSqGSIb3DQEDATBGAkEA9W/1UZ4P86AbmIyTO9iNlb5gO0txkxIPMe+f\nSb+BZlGsSXhHFB2jgjlHlhp5CzOwcbSkwIeA5g2Urfl4lOxQzwIBAgNEAAJBAIJL\nQt44JoJUEqjk2XmpSgQkJuU1qRXXB/vSipZQ5ryx/JLt40DEaPZdVDw4i3/GWOfd\nyLU1D8yDkEk1ZHUqQvk=\n-----END PUBLIC KEY-----\n'
2
Bob Public Key: b'-----BEGIN PUBLIC KEY-----\nMIGaMFMGCSqGSIb3DQEDATBGAkEA9W/1UZ4P86AbmIyTO9iNlb5gO0txkxIPMe+f\nSb+BZlGsSXhHFB2jgjlHlhp5CzOwcbSkwIeA5g2Urfl4lOxQzwIBAgNDAAJABghq\nT4r5EImoWVoKCzMJl3Z2Af/4NYOmxS/tQYnFtxsUXAF8ryWuiUDNNIMt3KhJTSKz\nxohAtKyI41ujS9g9kQ==\n-----END PUBLIC KEY-----\n'
2
Eave Public Key: b'-----BEGIN PUBLIC KEY-----\nMIGbMFMGCSqGSIb3DQEDATBGAkEA9W/1UZ4P86AbmIyTO9iNlb5gO0txkxIPMe+f\nSb+BZlGsSXhHFB2jgjlHlhp5CzOwcbSkwIeA5g2Urfl4lOxQzwIBAgNEAAJBAPU+\nU8Rjmr35mVWRgpyISZvD25UEa8TBrAia5EP4FAarh3JbuNuGlx3MBcFJ3KrD9y3Y\n6mHiJ4jEK7vzUAIqMqE=\n-----END PUBLIC KEY-----\n'


# Exchange
Now basically the game is for two actors to exchange their public keys and the third one to intersept them and eavesdrop the communication

In [63]:
alice_public_key = deserialize_public_key(alice_public_pem)
bob_public_key = deserialize_public_key(bob_public_pem)

# Eave intercepts the public keys and generates shared secrets with both Alice and Bob
eave_alice_shared_secret = eave_private_key.exchange(alice_public_key)

eave_bob_shared_secret = eave_private_key.exchange(bob_public_key)

print("Eave and Alice Shared Secret:", eave_alice_shared_secret)
print("Eave and Bob Shared Secret:", eave_bob_shared_secret)

Eave and Alice Shared Secret: b'^\x96m\xe4\x13t\x08\xb7\x7fri\x80\x91\x86\x98\xb4\xa6[\xbd<\xd6\x0e\xb4\x8a4^\x04\xc1\xab4\x14P\xc7\tzZH1\x15\x11a\xc3xG.\x92H\xc0\xc7\xec-!\xb4\xd8Eb\x86H\xb0gE\xbb\xd5\xb6'
Eave and Bob Shared Secret: b'\xb5\xa6\x9e\x1a\x83\xbcU\xa9W\xc5n\x8e\x94\xa9\x85\x8aN\xe9\x19\xf7\x1bD2\xd4\xbd5\xf7\xb7u\xed\xd4\r\x95~\xf2\xcc\x15\x98\xdb\xf4\x0bm1P\xb2\x04\xe3\xe0\xa0 \xfa\xf6W\xc5\x0c[F\x8fX\xb5\xdcu\xec\xe9'


# Key Derivation
Now Alice wants to send the shared secret to Bob and Bob wants to receive the shared secret from Alice.

Alice sends the shared secret to Bob
Eave intercepts the message, decrypts it, reencrypts it, and sends it to Bob

Bob receives the message from Eave.

The problem is that we need to derive a symmetric key from the shared secret as it cannot be used alone.

In [64]:
hkdf = HKDF(algorithm=hashes.SHA256(), length=32, salt=None, info=b"symmetric_key")
aes_key_alice_eave = hkdf.derive(eave_alice_shared_secret)
print(aes_key_alice_eave)
hkdf = HKDF(algorithm=hashes.SHA256(), length=32, salt=None, info=b"symmetric_key")
aes_key_bob_eave = hkdf.derive(eave_bob_shared_secret)
print(aes_key_bob_eave)

b'\xe5A\x06\x84\x05y\xb4\x16ns<0\x81\xfe\xa9\x16t\x12\xa9\x05\x07\xb6m\x80\xf8\xdf\xce\xae\xc2\xaa\x9d6'
b'\xf2\xb3\xd3\x9fe|\x0b\xa9\xaf\x0e\x8d\x0b\x9e\xc2\xfdY]AR\x17aXiF\xd3\xb9\xde\x84N\x18\nh'


In [65]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.padding import PKCS7
from cryptography.hazmat.backends import default_backend

# The message to be encrypted
message = b"Hi Bob, it's Alice's message"

def encrypt(key, message):
    # Padding the message to ensure it is a multiple of the block size
    padder = PKCS7(algorithms.AES.block_size).padder()
    padded_message = padder.update(message) + padder.finalize()

    # Create a new AES cipher with CBC mode
    cipher = Cipher(algorithms.AES(key), modes.CBC(b'IloveCryptografy'), backend=default_backend())

    # Encrypt the message
    encryptor = cipher.encryptor()
    return encryptor.update(padded_message) + encryptor.finalize()


alice_eave_encrypted = encrypt(aes_key_alice_eave, message)
print("Encrypted Message with AES key:", alice_eave_encrypted)


Encrypted Message with AES key: b'\x1d\t\xf4^\xa9\x88\x1dD4s\x8fp\xfe*&\xc5\x1d\xbeP\x15 5\x03\x97\xcf\xe8S\xcfyT\x1d\x99'


# Decrypt the message by Alice

In [69]:
# Assuming the derived key from HKDF is the AES key

# The encrypted message received
encrypted_message = alice_eave_encrypted
def decrypt(key, message):
    # Create a new AES cipher with CBC mode for decryption
    cipher = Cipher(algorithms.AES(key), modes.CBC(b'IloveCryptografy'), backend=default_backend())

    # Decrypt the message
    decryptor = cipher.decryptor()
    padded_decrypted_message = decryptor.update(message) + decryptor.finalize()

    # Unpadding the decrypted message to get the original message
    unpadder = PKCS7(algorithms.AES.block_size).unpadder()
    decrypted_message = unpadder.update(padded_decrypted_message) + unpadder.finalize()
    return decrypted_message

eave_alice_decrypted = decrypt(aes_key_alice_eave, encrypted_message)
print("Eave saw the message:", eave_alice_decrypted)


Eave saw the message: b"Hi Bob, it's Alice's message"


# Send message to Bob

In [70]:
# Decoding the message to remove the byte literal notation before concatenating with the new message
message = eave_alice_decrypted.decode("utf-8")
print("Alice's message to Bob:", message)
new_message = message + ". I need you to send me Bitcoin to this address: 1BoatSLRHtKNngkdXEeobR76b53LETtpyT"
print("New message to Bob:", new_message)

# Now sending that to Bob



Alice's message to Bob: Hi Bob, it's Alice's message
New message to Bob: Hi Bob, it's Alice's message. I need you to send me Bitcoin to this address: 1BoatSLRHtKNngkdXEeobR76b53LETtpyT


In [71]:
message_for_bob = encrypt(aes_key_bob_eave, new_message)

TypeError: data must be bytes-like